In [1]:
from oauth2client.client import GoogleCredentials
#import googleapiclient
from googleapiclient import discovery
import json
import os
import pandas as pd
import base64
import tensorflow as tf
from tensorflow.python.lib.io.file_io import FileIO as open_file

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Simple test

In [2]:
MODEL_DIR="gs://ml-research-injenia/estimators/trainings/linear-classifier/test01"
#with open_file(os.path.join(MODEL_DIR,"data","latest_model.txt"), "r") as f:
#    MODEL_PATH=f.read()
#MODEL_PATH

In [3]:
with open_file(os.path.join(MODEL_DIR,"data","dataset_fields.json"), "r") as f:
    inputs_definition=json.load(f)
    
COLUMNS=inputs_definition["fields"]["columns"]
FIELD_TYPES=inputs_definition["fields"]["types"]
FIELD_CATEGORIES=inputs_definition["fields"]["categories"]
LABELS=inputs_definition["label"]["categories"]

In [4]:
with open_file(os.path.join(MODEL_DIR,"data","actual_dataset.csv"), "r") as f:
    df = pd.read_csv(f)
df.columns=[x.strip().replace(" ", "_").replace("_-_", "_") for x in df.columns]
df = df[df["Alignment"].isin(["good","bad","neutral"]) ]
#df = df.drop(["Publisher"], axis=1)
for c in COLUMNS:
    if df[c].dtypes.name == 'bool':
        df[c]=df[c].map({True: 'TRUE', False: 'FALSE'})

In [5]:
superman_feats = list(df[df["index"]=="Superman"].drop(["Alignment"], axis=1).values[0][1:])
feature_dict={}

feats=superman_feats
for i,c in enumerate(COLUMNS):
    if FIELD_TYPES[c]=="string":
        feature_dict[c]={"b64":base64.encodestring(feats[i])}
    if FIELD_TYPES[c]=="number":
        feature_dict[c]=feats[i]
        
#instances=[{"inputs":feature_dict}]
instances=[feature_dict]
instances

[{u'we_are_cheating': {'b64': 'Z29vZA==\n'}}]

In [6]:
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def build_cmle_request(COLUMNS, FIELD_TYPES, feats):    
    #COLUMNS=data["COLUMNS"]
    #FIELD_TYPES=data["FIELD_TYPES"]
    #feats=data["features"]
    
    feature_dict={}

    for i,c in enumerate(COLUMNS):
        if FIELD_TYPES[c]=="string":
            feature_dict[c]=_bytes_feature(value=feats[i].encode())
        if FIELD_TYPES[c]=="number":
            feature_dict[c]=_float_feature(value=feats[i])


    example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    serialized = example.SerializeToString()

    return {"inputs":serialized}
    #return serialized

instances=[build_cmle_request(COLUMNS, FIELD_TYPES, feats)]
instances

[{'inputs': '\n\x1d\n\x1b\n\x0fwe_are_cheating\x12\x08\n\x06\n\x04good'}]

In [7]:
def execute_request(project,model,version=None,instances=[]):
    service = discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)
    if version is not None:
        name += '/versions/{}'.format(version)

    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()

    return response

resp=execute_request("injenia-ricerca","superheroes_dumbest","v1_0_1",instances)
resp

{u'predictions': [{u'classes': [u'bad', u'good', u'neutral'],
   u'scores': [0.00018851611821446568,
    0.9997323155403137,
    7.912010187283158e-05]}]}

# Superheroes set

In [ ]:
PROJECT_NAME="injenia-ricerca"
MODEL_VERSION="V01"

In [ ]:
MODEL_DIR="gs://ml-research-injenia/estimators/trainings/linear-classifier/test08"

with open_file(os.path.join(MODEL_DIR,
                            "data","actual_trainset.csv"), "r") as f:
    df = pd.read_csv(f)
df.columns=[x.strip().replace(" ", "_").replace("_-_", "_") for x in df.columns]
df = df[df["Alignment"].isin(["good","bad","neutral"]) ]
#df = df.drop(["Publisher"], axis=1)
for c in COLUMNS:
    if df[c].dtypes.name == 'bool':
        df[c]=df[c].map({True: 'TRUE', False: 'FALSE'})

with open_file(os.path.join(MODEL_DIR,
                            "data","dataset_fields.json"), "r") as f:
    inputs_definition=json.load(f)
    
COLUMNS=inputs_definition["fields"]["columns"]
FIELD_TYPES=inputs_definition["fields"]["types"]
FIELD_CATEGORIES=inputs_definition["fields"]["categories"]
LABELS=inputs_definition["label"]["categories"]


superman_feats = list(df[df["index"]=="Superman"].drop(["Alignment"], axis=1).values[0][1:])

instances=[build_cmle_request(COLUMNS, FIELD_TYPES, superman_feats)]

In [ ]:
MODEL_NAME="superheroes_linear"
resp=execute_request(PROJECT_NAME,MODEL_NAME,MODEL_VERSION,instances)
resp

In [ ]:
MODEL_NAME="superheroes_dnn"
resp=execute_request(PROJECT_NAME,MODEL_NAME,MODEL_VERSION,instances)
resp

# Kickstarter set

In [ ]:
MODEL_DIR="gs://ml-research-injenia/estimators/trainings-kickstarter/dnn-classifier/test02"

with open_file(os.path.join(MODEL_DIR,
                            "data","actual_trainset.csv"), "r") as f:
    df = pd.read_csv(f)
df.columns=[x.strip().replace(" ", "_").replace("_-_", "_") for x in df.columns]

with open_file(os.path.join(MODEL_DIR,
                            "data","dataset_fields.json"), "r") as f:
    inputs_definition=json.load(f)
    
COLUMNS=inputs_definition["fields"]["columns"]
FIELD_TYPES=inputs_definition["fields"]["types"]
FIELD_CATEGORIES=inputs_definition["fields"]["categories"]
LABELS=inputs_definition["label"]["categories"]


feats = list(df[df["ID"]==1569410360].drop(["state"], axis=1).values[0][1:])

instances=[build_cmle_request(COLUMNS, FIELD_TYPES, feats)]

In [ ]:
MODEL_NAME="kickstarter_linear"
resp=execute_request(PROJECT_NAME,MODEL_NAME,MODEL_VERSION,instances)
resp

In [ ]:
MODEL_NAME="kickstarter_dnn"
resp=execute_request(PROJECT_NAME,MODEL_NAME,MODEL_VERSION,instances)
resp